# How I did the HD setup

## 1. Check installed hard drives

`sudo fdisk -l`
```
Disk /dev/sda: 238,5 GiB, 256060514304 bytes, 500118192 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 512 bytes
I/O size (minimum/optimal): 512 bytes / 512 bytes
Disklabel type: gpt
Disk identifier: 9EA97655-0784-46C4-AB76-5C519C804862

Device        Start       End   Sectors  Size Type
/dev/sda1      2048  15624191  15622144  7,5G Linux swap
/dev/sda2  15624192 500117503 484493312  231G Linux filesystem


Disk /dev/sdb: 1,8 TiB, 2000398934016 bytes, 3907029168 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 4096 bytes
I/O size (minimum/optimal): 4096 bytes / 4096 bytes


Disk /dev/sdc: 1,8 TiB, 2000398934016 bytes, 3907029168 sectors
Units: sectors of 1 * 512 = 512 bytes
Sector size (logical/physical): 512 bytes / 4096 bytes
I/O size (minimum/optimal): 4096 bytes / 4096 bytes
```

Basically, the disks I want to setup, format and mount are `sdb` and `sdc`.
With the next I see that only `sda` is currently mounted:

```
df -h | grep -v /dev/loop

Filesystem      Size  Used Avail Use% Mounted on
udev             16G     0   16G   0% /dev
tmpfs           3,2G  2,4M  3,2G   1% /run
/dev/sda2       227G  160G   56G  75% /
tmpfs            16G  295M   16G   2% /dev/shm
tmpfs           5,0M  4,0K  5,0M   1% /run/lock
tmpfs            16G     0   16G   0% /sys/fs/cgroup
tmpfs           3,2G   28K  3,2G   1% /run/user/121
tmpfs           3,2G   88K  3,2G   1% /run/user/1003
```

## 2. Formatear y montar /dev/sdb

`sudo lshw -C disk`

I will copy just the info for `sdb`

```
*-disk
       description: ATA Disk
       product: ST2000DM001-1ER1
       vendor: Seagate
       physical id: 0.0.0
       bus info: scsi@1:0.0.0
       logical name: /dev/sdb
       version: CC28
       serial: Z4ZBQY1R
       size: 1863GiB (2TB)
       configuration: ansiversion=5 logicalsectorsize=512 sectorsize=4096
```

Format the disk using `parted`.
First, open the `parted` console:

```
sudo parted /dev/sdb
[sudo] password for cantalapiedra: 
GNU Parted 3.2
Using /dev/sdb
Welcome to GNU Parted! Type 'help' to view a list of commands.
(parted) 
```

Within `parted`, create the partition, then quit:
```
(parted) mklabel gpt                                                      
(parted) unit TB                                                          
(parted) mkpart primary 0.00TB 2.00TB
(parted) print                                                            
Model: ATA ST2000DM001-1ER1 (scsi)
Disk /dev/sdb: 2,00TB
Sector size (logical/physical): 512B/4096B
Partition Table: gpt
Disk Flags: 

Number  Start   End     Size    File system  Name     Flags
 1      0,00TB  2,00TB  2,00TB               primary

(parted) quit

```

Format the partition:
```
sudo mkfs -t ext4 /dev/sdb1
mke2fs 1.44.1 (24-Mar-2018)

Creating filesystem with 488378368 4k blocks and 122101760 inodes
Filesystem UUID: ee775237-2607-44dd-870c-399b8885586d
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000, 7962624, 11239424, 20480000, 23887872, 71663616, 78675968, 
	102400000, 214990848

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (262144 blocks): done
Writing superblocks and filesystem accounting information: done
```

Create the directory where the HD will be mounted (I chose "/data" but you could choose any name you want):
```
sudo mkdir /data
```

Mount the HD:

```
sudo mount /dev/sdb1 /data
sudo chown -R USERNAME:GROUPNAME /data # change the owner of the directory to your user and group)
sudo chmod 755 /data  ## make sensible permissions
cd /data              ## change to the mount directory
ls -a                  ## list content of the new hard drive
```

Unmount the HD (because we will use automount through `/etc/fstab`):
```
sudo umount /dev/sdb1
```


Retrieve the UUID:
```
sudo blkid | grep sdb1
/dev/sdb1: UUID="d7618b9c-614a-4f55-8de3-78c6e51ac9c0" TYPE="ext4" PARTLABEL="primery" PARTUUID="bd2bab25-a985-4882-82e9-d3ac489f19a0"
```

Add mount to `/etc/fstab`
```
sudo cp /etc/fstab /etc/fstab.bak # backup the file (just in case!)
sudo nano /etc/fstab # (open the file) and add the next line:
UUID=d7618b9c-614a-4f55-8de3-78c6e51ac9c0   /data    ext4    defaults    0    2
```

After `sudo mount -a` we should see the files within `/data` with `ls /data`

In [3]:
%%HTML
<img src="http://e00-marca.uecdn.es/assets/multimedia/imagenes/2016/10/16/14766103230639.jpg"\>

## postdata

Por si en el futuro hay que hacerlos RAID, un enlace con pasos sugeridos es este: 

[Setup](Setup existing drive as RAID.ipynb)

ó

https://wiki.archlinux.org/index.php/Convert_a_single_drive_system_to_RAID